In [4]:
# 1. Setup Environment Variables and Imports
import os
from dotenv import load_dotenv
from langsmith import traceable # Required for tracing!
from typing import List
import nest_asyncio
# We use ChatGroq instead of OpenAI
from langchain_groq import ChatGroq 
from utils import get_vector_db_retriever, RAG_SYSTEM_PROMPT

# Load environment variables from .env file
load_dotenv()

MODEL_PROVIDER = "groq"
MODEL_NAME = "llama-3.3-70b-versatile" # Using a fast, standard Groq model

# Initialize Groq client (LangChain handles authentication via ENV variable)
llm_client = ChatGroq(model=MODEL_NAME)
nest_asyncio.apply()
retriever = get_vector_db_retriever()

In [5]:
# 2. Apply @traceable decorators with custom metadata (YOUR TWEAK)

@traceable(
    name="Retrieve Documents",
    metadata={"vectordb_type": "FAISS", "k_neighbors": 2} # TWEAK: Adding descriptive retrieval metadata
)
def retrieve_documents(question: str):
    return retriever.invoke(question)   


@traceable(name="Generate Response Chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    # Call the Groq LLM client
    response = llm_client.invoke(messages)
    return response


@traceable(
    name="Final LLM Call",
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER} # TWEAK: Adding descriptive LLM metadata
)
def call_llm(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
):
    # The actual LLM invocation happens here and is traced automatically by LangChain,
    # but we keep this function traced to show the metadata context.
    return llm_client.invoke(messages)


@traceable(name="Root RAG Pipeline")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    # Use call_llm to show the context in the trace
    response = call_llm(
        [
            {"role": "system", "content": RAG_SYSTEM_PROMPT},
            {"role": "user", "content": f"Question: {question}. Context: {documents}"}
        ]
    )
    return response.content

In [6]:
# 3. Execute the pipeline and view the trace in Langsmith
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)

print("--- Question ---\n")
print(question)
print("\n--- Answer ---\n")
print(ai_answer)

question_metadata = "How do I add Metadata to a Run with @traceable?"
ai_answer_metadata = langsmith_rag(question_metadata)

print("\n--- Question (Metadata Check) ---\n")
print(question_metadata)
print("\n--- Answer ---\n")
print(ai_answer_metadata)

--- Question ---

How can I trace with the @traceable decorator?

--- Answer ---

To trace with the @traceable decorator, you can add it to any function to automatically track its inputs, outputs, and errors in LangSmith. You can also pass a 'metadata' dictionary to the @traceable decorator to add metadata to traces. Additionally, metadata can be added at runtime using the 'langsmith_extra' parameter.

--- Question (Metadata Check) ---

How do I add Metadata to a Run with @traceable?

--- Answer ---

To add metadata to a run with @traceable, you can pass a 'metadata' dictionary to the @traceable decorator. This allows you to include additional information with your traces. You can also use the 'langsmith_extra' parameter to add metadata at runtime.
